In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

np.random.seed(123)
torch.manual_seed(123)

In [9]:
# Custom loss function
margin = 1
class SquaredHingeLoss(nn.Module):
    def forward(self, predicted, low, high):
        low  = low + margin
        high = high - margin
        loss = torch.relu(low - predicted) + torch.relu(predicted - high)
        return torch.sum(loss**2)

In [10]:
# # train data
# data_df   = pd.read_csv('data/seq_stat.csv')
# target_df = pd.read_csv('record/data_fold1.csv')

# data      = data_df.iloc[:, 1:].to_numpy()
# targets_1 = target_df.iloc[:, 1:2].to_numpy()
# targets_2 = target_df.iloc[:, 2:3].to_numpy()

# data      = torch.FloatTensor(data)
# targets_1 = torch.FloatTensor(targets_1)
# targets_2 = torch.FloatTensor(targets_2)

# # test_data
# target_test_df = pd.read_csv('record/data_fold2.csv')
# targets_test_1 = target_test_df.iloc[:, 1:2].to_numpy()
# targets_test_2 = target_test_df.iloc[:, 2:3].to_numpy()

# targets_test_1 = torch.FloatTensor(targets_test_1)
# targets_test_2 = torch.FloatTensor(targets_test_2)

In [11]:
def normalize_data(data):
    mean = data.mean(dim=0)
    std  = data.std(dim=0)
    norm_data = (data - mean) / std
    return norm_data

In [12]:
data_df   = pd.read_csv('learning_data/seq_features.csv')
target_df = pd.read_csv('learning_data/target_lambda_bothfold.csv')

n_train = 300

# train data
data = torch.FloatTensor(data_df.iloc[:n_train, 1:].to_numpy())
data = normalize_data(data)
targets_1 = torch.FloatTensor(target_df.iloc[:n_train, 1:2].to_numpy())
targets_2 = torch.FloatTensor(target_df.iloc[:n_train, 2:3].to_numpy())

# test_data
data_test = torch.FloatTensor(data_df.iloc[n_train:, 1:].to_numpy())
data_test = normalize_data(data_test)
targets_test_1 = torch.FloatTensor(target_df.iloc[n_train:, 1:2].to_numpy())
targets_test_2 = torch.FloatTensor(target_df.iloc[n_train:, 2:3].to_numpy())

In [13]:
# Define the deep learning model
class MyModel(nn.Module):
    def __init__(self, input_size):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 8)
        self.fc2 = nn.Linear(8, 4)
        self.fc3 = nn.Linear(4, 2)
        self.fc4 = nn.Linear(2, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [14]:
# Instantiate the model, define custom loss function, and optimizer
model = MyModel(input_size = data_df.shape[1] - 1)
squared_hinge_loss = SquaredHingeLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10000
for epoch in range(epochs+1):
    # Forward pass
    outputs_train = model(data)
    outputs_test  = model(data_test)
    
    # Compute the custom loss
    loss_train = squared_hinge_loss(outputs_train, targets_1, targets_2)
    loss_test  = squared_hinge_loss(outputs_test, targets_test_1, targets_test_2)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    
    # Print the loss every 100 epochs
    if (epoch) % 500 == 0:
        print(f'Epoch [{epoch:5d}/{epochs}], Loss_train: {loss_train.item():8.4f}, Loss_test: {loss_test.item():8.4f}')

Epoch [    0/10000], Loss_train: 686.7583, Loss_test: 249.6145


Epoch [  500/10000], Loss_train: 537.1819, Loss_test: 170.8680
Epoch [ 1000/10000], Loss_train: 531.4022, Loss_test: 173.6916
Epoch [ 1500/10000], Loss_train: 526.9100, Loss_test: 174.2226
Epoch [ 2000/10000], Loss_train: 522.7416, Loss_test: 177.2994
Epoch [ 2500/10000], Loss_train: 518.2928, Loss_test: 181.1276
Epoch [ 3000/10000], Loss_train: 514.1561, Loss_test: 186.8445
Epoch [ 3500/10000], Loss_train: 511.5004, Loss_test: 192.3344
Epoch [ 4000/10000], Loss_train: 508.8043, Loss_test: 199.8932
Epoch [ 4500/10000], Loss_train: 506.6811, Loss_test: 210.6202
Epoch [ 5000/10000], Loss_train: 503.7804, Loss_test: 236.4378
Epoch [ 5500/10000], Loss_train: 502.3000, Loss_test: 245.6436
Epoch [ 6000/10000], Loss_train: 501.4751, Loss_test: 251.4688
Epoch [ 6500/10000], Loss_train: 500.8625, Loss_test: 256.1204
Epoch [ 7000/10000], Loss_train: 500.4200, Loss_test: 258.6102
Epoch [ 7500/10000], Loss_train: 500.0414, Loss_test: 262.3310
Epoch [ 8000/10000], Loss_train: 499.7447, Loss_test: 2